In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
import pandas as pd
import random
import numpy as np

from tqdm import tqdm
from torch import nn

/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-04-20 23:47:37,632] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
    # sentence dataset struct
    class SentenceDataset(Dataset):
        def __init__(self, tokenizer, file_path, max_len, lan_codes=None, split=None):
            self.tokenizer = tokenizer
            self.sentences = pd.read_csv(file_path)
            train_ratio = 0.999
    
            if lan_codes is not None:
                # filter languages
                self.sentences = self.sentences[self.sentences['lan_code'].isin(lan_codes)]
            if split is not None:
                length = len(self.sentences)
                if split == 'train':
                    self.sentences = self.sentences.iloc[:int(length * train_ratio)]
                elif split == 'test':
                    self.sentences = self.sentences.iloc[int(length * train_ratio):]
                else:
                    raise ValueError(f"split {split} not recognized")
            self.max_len = max_len
            
            print(f"Done loading dataset for split {split}")
    
        def __len__(self):
            return len(self.sentences)
    
        def __getitem__(self, item):
            sentence = str(self.sentences.iloc[item]['sentence']).lower()
            encoding = self.tokenizer(sentence, max_length=self.max_len, padding='max_length', truncation=True)
            return {key: torch.tensor(val) for key, val in encoding.items()}

In [3]:
# training arguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def convert_weights_to_fp16(model: nn.Module, dtype=torch.bfloat16):
    """Convert applicable model parameters to fp16"""

    def _convert_weights_to_fp16(l):
        if isinstance(l, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            l.weight.data = l.weight.data.to(dtype)
            if l.bias is not None:
                l.bias.data = l.bias.data.to(dtype)

    model.apply(_convert_weights_to_fp16)

def reinitialize_weights(model):
    for module in model.modules():
        if isinstance(module, torch.nn.Linear):
            module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, torch.nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, torch.nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

In [4]:
args = {
    'batch_size': 60,
    'lr': 2e-5,
    'log_freq': 1,
    'eval_freq': 100,
    'epochs': 3,
    'mlm_prob': 0.4,
    'grad_accums': 20
}
lan_code = 'eng'

In [5]:


lan_codes = [lan_code]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = SentenceDataset(tokenizer, 'data/big-language-detection/sentences.csv', max_len=512, split='train')
test_dataset = SentenceDataset(tokenizer, 'data/big-language-detection/sentences.csv', max_len=512, split='test')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size*3,
                             shuffle=True,
                             collate_fn=data_collator)

Done loading dataset for split train
Done loading dataset for split test


In [6]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
reinitialize_weights(model)    # randomize weights so we start from scratch
convert_weights_to_fp16(model)
model.to(device)
print(f"training with {sum([p.numel() for p in model.parameters() if p.requires_grad]) / 1e6:.2f}M parameters")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


training with 109.51M parameters


In [7]:
import wandb

wandb.init(project='ling')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pranavputta-tonic (cvmlp-embodiment-transfer). Use `wandb login --relogin` to force relogin


In [ ]:

optimizer = AdamW(model.parameters(), lr=lr)

def train_step(model, batch, optimizer, step_optim):
    inputs = {k: v.to(device) for k, v in batch.items()}
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs = model(**inputs)
    loss = outputs.loss
    loss.backward()
    if step_optim:
        optimizer.step()
        optimizer.zero_grad()
    return loss

@torch.no_grad()
def eval_step(model):
    # step has different meanings here lol but whatever
    total_loss = 0
    for batch in tqdm(test_dataloader, desc='Eval', total=len(test_dataloader)):
        inputs = {k: v.to(device) for k, v in batch.items()}
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(**inputs)
        total_loss += outputs.loss
    return total_loss / len(test_dataloader)

def train(args):
    # train
    lr = args['lr']
    batch_size = args['batch_size']
    log_freq = args['log_freq']
    eval_freq = args['eval_freq']
    ckpt_freq = args['ckpt_freq']
    epochs = args['epochs']
    mlm_prob = args['mlm_prob']
    grad_accums = args['grad_accums']
    
    step = 0
    for epoch in (range(epochs)):  # Number of training epochs
        model.train()
        train_loss = 0.0
        for i, batch in tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch}", total=len(train_dataloader)):
            if (i + 1) % grad_accums != 0:
                train_loss += train_step(model, batch, optimizer, step_optim=False)
                continue
       
            train_loss += train_step(model, batch, optimizer, step_optim=True)
            train_loss /= grad_accums
            step += 1                

            if (step) % eval_freq == 0:
                test_loss = eval_step(model)
                print("Eval Loss:" ,test_loss)
                wandb.log({"test.loss": test_loss})
    
            if (step) % log_freq == 0:
                print(f"Loss: {train_loss}")
                wandb.log({"train.loss": train_loss})

            if (step) % ckpt_freq == 0:
                os.makedirs("checkpoints", exist_ok=True)
                torch.save(f"checkpoints/{lan_code}_{step}.pt")

            train_loss = 0.

/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 0:   0%|                                                                                                                                | 20/172192 [00:20<52:25:22,  1.10s/it]

Loss: 10.50524616241455


Epoch 0:   0%|                                                                                                                                | 40/172192 [00:40<52:09:07,  1.09s/it]

Loss: 10.364455223083496


Epoch 0:   0%|                                                                                                                                | 60/172192 [00:59<52:10:57,  1.09s/it]

...

Epoch 0:   1%|█▌                                                                                                                            | 2080/172192 [35:06<51:35:01,  1.09s/it]

Loss: 9.022245407104492


Epoch 0:   1%|█▌                                                                                                                            | 2100/172192 [35:25<51:34:50,  1.09s/it]

Loss: 8.965218544006348


Epoch 0:   1%|█▌                                                                                                                            | 2111/172192 [35:36<46:22:28,  1.02it/s]

In [ ]:
torch.cuda.is_available()

In [ ]:
dataset.sentences.iloc[:100]